# Inference
Get predictions for submission

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pretrainedmodels
import pandas as pd
import os

from tools import *


In [ ]:
device = get_device()

## Importing label dataset

To run the full crossvalidation, remove .head(30000). This will multiply about by 20 the time required by this notebook.

In [ ]:

# FOLDERS PATH
source_dir = 'histopathologic-cancer-detection/'
test_im_source_dir = source_dir+'/test'

# IMPORTING DATA
test_data = pd.read_csv(os.path.join(source_dir,'test_labels.csv')) 



In [ ]:
data_ids = list(test_data.id)
data_labels = list(test_data.label)

## Loading model

In [ ]:
model_source_dir = 'trained_model/'
model_name = 'final'

model_path =  model_source_dir+'/{}.model.pt'.format(model_name)


In [ ]:
# Import pretrained model
base_model = pretrainedmodels.resnet34(num_classes=1000, 
                                    pretrained='imagenet').to(device) 
    
# Shape the model    
model = Net(base_model, 512).to(device)

# Load model from saved 
model.load_state_dict(torch.load(model_path))



## Inference

In [ ]:
test_loader = torch.utils.data.DataLoader(DataGenerator(
                                                data_ids, 
                                                data_labels, 
                                                aug_val(), 
                                                test_im_source_dir), 
                                            shuffle=False, 
                                            pin_memory=False, 
                                            num_workers=1,
                                            batch_size=1) 

model.eval() 

predictions = []
batch_prediction = []

with torch.no_grad():
    for batch_idx, (x, target) in enumerate(test_loader):
        # Test time augmentation
        image = np.rollaxis(x.numpy()[0], 0, 3)
        images = produce_test_time_augmentation(image,n_images=8)
        
        output = model(torch.from_numpy(images).to(device, dtype=torch.float))
        output = output.mean()            
        #output = protein_model(x.to(device, dtype=torch.float))
        batch_prediction[batch_idx] = output
    
    predictions.append(batch_prediction)    
    
    

predictions = np.vstack(predictions) # get together the batches (one batch contains tta of one image)
predictions = np.mean(predictions, axis=0) # mean of tta of one image


test_data['label'] = predictions

test_data.to_cvs('predictions.csv', sep =',', index = False)